# Import Libraries


In [58]:
import pandas as pd # data processing
import numpy as np # Mathematical operations
import seaborn as sns # Data visualization
import matplotlib.pyplot as plt # Data visualization
import moviepy.editor as mpy # Video and audio editing
import requests # HTTP requests
import os # File operations
import sys # System operations
import boto3 # AWS SDK
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pydub import AudioSegment # Audio processing

# Media Files

    A. Issues when loading the video: 

                1. When previewing the video it has a mixer issue - Will skip  this for now till fix is found, will convert the file and store to bucket
                2. Where do we store the temporary files - Files will be stored in a bucket on S3 on AWS

**File convertion**

In [ ]:
file_path = "C:/Users/tedjo/Downloads/dancevibedaily/media/"
save_path = "C:/Users/tedjo/Downloads/dancevibedaily/media/audio/"

#Loop through each file within the directory
for file in os.listdir(file_path):
    if file.endswith(".mp4"):
        video_path = os.path.join(file_path, file)

        #Load video file
        try:
            video = mpy.VideoFileClip(video_path)
            print(f'Video Loaded: {video_path}')
            audio = video.audio
            audio.write_audiofile(os.path.join(save_path,f'{file}.mp3'))
            
            # #Access AWS S3 Bucket
            # aws_access_key_id = ''
            # aws_secret_access_key = ''

            # s3 = boto3.resource('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

            # with open(os.path.join(file_path,f'{file}.mp3'), 'rb') as data:
            #     s3.Bucket('dancevibedaily').put_object(Key=f'{file}.mp3', Body=data)
            #     print(f'Audio file uploaded to S3: {file}.mp3')

            # #Remove audio file from local storage
            # os.remove(os.path.join(file_path,f'{file}.mp3'))
            # print("Audio file removed from local storage")

        except Exception as e:
            print(f"Error loading video: {e}")
            continue

# Data Building and Cleaning

**Issues on API use with bucket**

                1. As I am new to Data Engineering I have not figured out how to work with api for now will work with local directory.
                2. There is a 'NoneType' object is not subscriptable, will go through these json and get to understand the issue.
                3. Will need to rename the media file to match the song and data in the uri, to be able to identify track 
                   1. To deal with this I have created a uri dataframe which will hold the mp3 file name and title of track
                   2. This will help in finding:
                      1. Tracks that may not have a match
                      2. Match the number of likes to the song 

**Audd API**

In [75]:
import requests # HTTPS Requests
data = {
    'api_token': '1f11b47922b072fe40304ce3cefd8c69',
    'return': 'musicbrainz',
}

# Will loop through the files in the directory and extract the song title, artist, and album from the API response
save_path = "C:/Users/tedjo/Downloads/dancevibedaily/media/audio/"
results = [] # List to store the results of the API response
uri_song = [] # Reel JSON URI and the Title of the song

for i in os.listdir(save_path):
    if i.endswith(".mp3"):
        files = {
            'file': open(os.path.join(save_path,i),'rb') #reading binary file
        }
        try:
            result = requests.post('https://api.audd.io/', data=data, files=files) # Send data to the API 
            if result.json()['status'] == 'error':
                print(f"Error processing file: {result.json()}")
                continue
            print(result.json()['result']['title'])
            results.append({
                'Title': result.json()['result']['title'],
                'Artist': result.json()['result']['artist'],
                'Album': result.json()['result']['album'],
                'Release Date': result.json()['result']['release_date'],
            })# Append the results to the list, which will be used to create a DataFrame
            uri_song.append({
                'uri': i,
                'Title': result.json()['result']['title'],
            })
        except Exception as e:
            print(f"Error processing file: {e}")
            continue

df = pd.DataFrame(results) # Create a DataFrame from the results list
df_1 = pd.DataFrame(uri_song)


All Over You
ULAZI Infinity MusiQ (yey)
Tjina
YESHUA (dance cruise)
Private school
Error processing file: 'NoneType' object is not subscriptable
Look at Her
Killin Dem
Error processing file: 'NoneType' object is not subscriptable
Escapism. (Sped Up)
Error processing file: 'NoneType' object is not subscriptable
Error processing file: 'NoneType' object is not subscriptable
Jinja
Dalie (feat. Baby S.O.N)
soso
الي اين انا ذاهب
Water
Issa Necessary (feat. Kiss Daniel)
Mnike (UK Radio Edit)
Barbie World (with Aqua) [From Barbie The Album]
UNAVAILABLE (feat. Musa Keys)
Look at Her
Hwiralang feat. Siko Wa Mmino,Hlogi Mash
dabala
Mnike
Cough
Error processing file: 'NoneType' object is not subscriptable
Water
Venda Boy (feat. fefe laku dura)
AH TXE TXE (feat. TYSON)
Floor Seats
Error processing file: 'NoneType' object is not subscriptable
Rock Attack
Walking dead
Kafe'
Shu! (feat. Chley)
Error processing file: 'NoneType' object is not subscriptable
Mnike
Get Up (Main Version)
Lost In The Bass. M

In [76]:
uri_song

[{'uri': '034307877B906E78461292BF61ADEDBC_video_dashinit_18046290733490860.mp4.mp3',
  'Title': 'All Over You'},
 {'uri': '034377C77CC9159980937FFB85869E91_video_dashinit_18067660132487904.mp4.mp3',
  'Title': 'ULAZI Infinity MusiQ (yey)'},
 {'uri': '0346D7295270CA495F2EADA738BC25BB_video_dashinit_18087606337378512.mp4.mp3',
  'Title': 'Tjina'},
 {'uri': '034D9E4D23B63222BDAA18EC81AB14B1_video_dashinit_18369057538017056.mp4.mp3',
  'Title': 'YESHUA (dance cruise)'},
 {'uri': '0447ECEF8BC23236B2795670E21CA784_video_dashinit_18062114641477365.mp4.mp3',
  'Title': 'Private school'},
 {'uri': '05431CA586A3D30961CA3A282FFD7B95_video_dashinit_17864735624927121.mp4.mp3',
  'Title': 'Look at Her'},
 {'uri': '0547CA0F2571F2A6098600935DF4AAA3_video_dashinit_17856200351919402.mp4.mp3',
  'Title': 'Killin Dem'},
 {'uri': '0849B3054054CB35EC9B576781A4AF86_video_dashinit_18039911848431831.mp4.mp3',
  'Title': 'Escapism. (Sped Up)'},
 {'uri': '0B4E652644469C6842418E0B752CAEAC_video_dashinit_18240419

In [77]:
df.to_csv('C:/Users/tedjo/Downloads/dancevibedaily/media/audio/audio_metadata.csv', index=False) # Save the DataFrame to a CSV file
df_1.to_csv('C:/Users/tedjo/Downloads/dancevibedaily/media/audio/uri_reel_data.csv', index=False)

In [79]:
df

,Title,Artist,Album,Release Date
0,All Over You,Sped Up 204,GORGEOUS,2023-09-08
1,ULAZI Infinity MusiQ (yey),uLazi,ULAZI Infinity MusiQ (yey),2022-12-01
2,Tjina,Megadrumz/Lady Du,For Your Soul (Extended Edition),2023-09-01
3,YESHUA (dance cruise),Dj Bentoa,YESHUA (dance cruise),2023-04-22
4,Private school,Juicy Kay Dee,Private school,2021-02-18
...,...,...,...,...
852,Ka Valungu,Tebza De DJ,Ka Valungu,2023-07-26
853,No Wahala,Speedy Songs,No Wahala,2022-06-22
854,MY POWER,Nija,The Lion King: The Gift,2019-07-19
855,Yini Ngathi (Insane) (Live),Felo Le Tee,Yini Ngathi,2023-10-13


**Spotify API**

In [65]:
client_id = ''# Spotify Client Token
client_secret = ''  # Spotify Secret Token
auth_manager = SpotifyClientCredentials(client_id = client_id, client_secret=client_secret)# Initialize the Spotify Client Credentials
sp = spotipy.Spotify(auth_manager=auth_manager)# Initialize the Spotify API client

# #New Columns
# df['Spotify Track ID'] ='' # Track ID for other analysis on tracks
# df['Spotify Track Name'] = ''# Track Name
# df['Sporify Album'] = '' # Album Name
# df['Spotify Arist'] = '' # Artists Name
# df['Spotify Track URL'] = '' # Track URL
# df['Spotify Track Genre'] = ''#Track Genre
# df['Spotify Image Path'] = ''#Image Path
# df['Spotify Track Popularity'] = ''#How popular is this track from 0 to 100

#Function to get details
def get_details(track_name,artist_name,album_name):
    query = f'track: {track_name} artist: {artist_name} album: {album_name}'
    result  = sp.search(q= query, type= 'track',limit=1 )
    if result['tracks']['items']:
        track = result['tracks']['items'][0]
        artist_id = track['artists'][0]['id']
        artist = sp.artist(artist_id)
        return{
            'Spotify Track ID': track['id'],
            'Spotify Track Name': track['name'],
            'Spotify Artist': ', '.join([artist['name'] for artist in track['artists']]),
            'Spotify Album': track['album']['name'],
            'Spotify URL': track['external_urls']['spotify'],
            'Spotify Image URL': track['album']['images'][0]['url'] if track['album']['images'] else None,
            'Spotify Track Popularity': track['popularity'],
            'Spotify Track Genre': ', '.join(artist['genres'])  if artist['genres'] else None
        }
    else:
        return None

for index, row in df.iterrows():
    details = get_details(row['Title'],row['Artist'],row['Album'])
    print(f"Processing {index+1} of {len(df)}")
    print(details)
    if details:
        df.at[index, 'Spotify Track ID'] = details['Spotify Track ID']
        df.at[index, 'Spotify Track Name'] = details['Spotify Track Name']
        df.at[index, 'Spotify Artist'] = details['Spotify Artist']
        df.at[index, 'Spotify Album'] = details['Spotify Album']
        df.at[index, 'Spotify Track URL'] = details['Spotify URL']
        df.at[index, 'Spotify Image URL'] = details['Spotify Image URL']
        df.at[index, 'Spotify Track Popularity'] = details['Spotify Track Popularity']
        df.at[index, 'Spotify Track Genre'] = details['Spotify Track Genre']


Processing 1 of 195
{'Spotify Track ID': '75Q69chmd8CEZbVsA4CDMm', 'Spotify Track Name': 'Gorgeous', 'Spotify Artist': 'mansionz', 'Spotify Album': 'Mansionz', 'Spotify URL': 'https://open.spotify.com/track/75Q69chmd8CEZbVsA4CDMm', 'Spotify Image URL': 'https://i.scdn.co/image/ab67616d0000b27390d0e1eb630b1595d23a1a0e', 'Spotify Track Popularity': 56, 'Spotify Track Genre': 'indie pop rap'}
Processing 2 of 195
{'Spotify Track ID': '4k2yKSDE5cSUED46GFbSdd', 'Spotify Track Name': 'ULAZI Infinity MusiQ (yey)', 'Spotify Artist': 'uLazi', 'Spotify Album': 'ULAZI Infinity MusiQ (yey)', 'Spotify URL': 'https://open.spotify.com/track/4k2yKSDE5cSUED46GFbSdd', 'Spotify Image URL': 'https://i.scdn.co/image/ab67616d0000b2739a1373a5bb0ce5db2c080155', 'Spotify Track Popularity': 28, 'Spotify Track Genre': 'sgija'}
Processing 3 of 195
{'Spotify Track ID': '4DO34AAYHCCF2gs7jxLFYx', 'Spotify Track Name': 'Tjina', 'Spotify Artist': 'Megadrumz, Lady Du', 'Spotify Album': 'For Your Soul', 'Spotify URL': 'h

# Data Cleaning